**TODO :**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/foursquare/src


In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# **Librairies**

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

<IPython.core.display.Javascript object>

In [12]:
import gc
import cudf
import random
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pandarallel import pandarallel

from params import DEBUG, OUT_PATH, IS_TEST, RESSOURCES_PATH
from ressources import *
from matching import *
from dtypes import DTYPES_1
from data.preparation import reduce_mem_usage

pandarallel.initialize(nb_workers=12, progress_bar=False)
random.seed(13)
warnings.simplefilter("ignore")
pd.options.display.max_columns = 500

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


<IPython.core.display.Javascript object>

## Load Data
- TODO: load precomputed probas

In [6]:
if IS_TEST:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_test.csv")
else:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")

<IPython.core.display.Javascript object>

In [7]:
train["idx"] = np.arange(len(train))

<IPython.core.display.Javascript object>

In [8]:
THRESHOLD = 0.0075

<IPython.core.display.Javascript object>

In [13]:
if IS_TEST:
    df_p = pd.read_csv(
        OUT_PATH + f"features_test_1_filtered_{THRESHOLD}.csv", dtype=DTYPES_1
    )
    SIZE_RATIO = 1
else:
    df_p = pd.read_csv(
        OUT_PATH + f"features_train_1_filtered_{THRESHOLD}.csv", dtype=DTYPES_1
    )
    SIZE_RATIO = 1

<IPython.core.display.Javascript object>

In [15]:
DEBUG = True

<IPython.core.display.Javascript object>

In [16]:
if DEBUG:
    df_p = df_p.head(10000).copy()

<IPython.core.display.Javascript object>

In [17]:
# df_merged = feature_engineering_2(df_p, train, ressources_path=RESSOURCES_PATH)

<IPython.core.display.Javascript object>

### Youri & Vincent

In [18]:
from fe import FE2

<IPython.core.display.Javascript object>

In [19]:
df_p = df_p.merge(train[["id", "Nb_multiPoi"]], left_on="id_1", right_on="id").drop(
    "id", axis=1
)
df_p = df_p.merge(
    train[["id", "Nb_multiPoi"]], left_on="id_2", right_on="id", suffixes=("_1", "_2")
).drop("id", axis=1)

<IPython.core.display.Javascript object>

In [20]:
p1 = df_p[["id_1"]].copy()
p1.columns = ["id"]
p2 = df_p[["id_2"]].copy()
p2.columns = ["id"]

<IPython.core.display.Javascript object>

In [21]:
%%time
df = FE2(df_p.copy(), p1, p2, train, RESSOURCES_PATH, size_ratio=SIZE_RATIO)

- Cat links & quantiles
- Simply connected components
- Strongly connected components
- Cat link score
- Link between grouped categories
- Features for column name_initial
- Features for column name_initial_decode
- Features for column nameC
- Features for column name
- Features for column categories
- Features for column address
- Features for column url
- Features for column city
- Features for column state
- Features for column zip
- Features for column phone
- Count encoding
- Words in categories
- Words in names
- Close count
- Close count of same category2
- Close count of same category_simpl
- Compare numeric part of the name/address
- Number in names features
CPU times: user 14.7 s, sys: 30.5 s, total: 45.2 s
Wall time: 58.6 s


<IPython.core.display.Javascript object>

### Théo

In [22]:
from fe_theo import feature_engineering_theo_2

<IPython.core.display.Javascript object>

In [23]:
cols = [
    "id",
    "name",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "state",
    "zip",
    "idx",
]

for col in cols[1:]:
    train.loc[train[col] == "", col] = np.nan

p1 = p1[["id"]].merge(train[cols], on="id", how="left")
p2 = p2[["id"]].merge(train[cols], on="id", how="left")

pairs = pd.concat([p1[cols], p2[cols]], axis=1)
pairs.columns = [c + "_1" for c in cols] + [c + "_2" for c in cols]

<IPython.core.display.Javascript object>

In [24]:
%%time

df_theo, fts_theo = feature_engineering_theo_2(train.copy(), pairs.copy(), cuda=False)

- Computing nan features
- Computing feature name_tf_idf_33_char_wb_sim
- Computing feature address_tf_idf_33_char_wb_sim
- Computing feature url_tf_idf_33_char_wb_sim
- Column : name  -  Function : wratio
- Column : name  -  Function : partial_ratio
- Column : address  -  Function : wratio
- Column : address  -  Function : partial_ratio
- Column : url  -  Function : wratio
- Column : url  -  Function : partial_ratio
CPU times: user 20.4 s, sys: 3.62 s, total: 24.1 s
Wall time: 25.2 s


<IPython.core.display.Javascript object>

## Merge & Save

In [25]:
df_merged = df.merge(df_theo, on=["id_1", "id_2"])

<IPython.core.display.Javascript object>

In [26]:
cols_to_end = [
    "point_of_interest_1",
    "fold_1",
    "point_of_interest_2",
    "fold_2",
    "match",
]
cols_to_end = [c for c in cols_to_end if c in df_merged.columns]

if len(cols_to_end):
    to_end = df_merged[cols_to_end]
    df_merged.drop(cols_to_end, axis=1, inplace=True)
    df_merged[cols_to_end] = to_end

<IPython.core.display.Javascript object>

In [27]:
if DEBUG:
    corrs = df_merged.corr()
    for col in corrs.columns:
        close = pd.DataFrame(corrs.loc[corrs[col] > 0.99][col])
        if len(close) > 1:
            display(close[close[col] < 1])

,angular_distance_min
angular_distance_l2_min,0.992907


,angular_distance_l2_min
angular_distance_min,0.992907


,url_len_diff
url_ld,0.99312


,ratio_connect_multipoi1
ratio_strong_connect_multipoi1,0.996524


,ratio_connect_multipoi2
ratio_strong_connect_multipoi2,0.990887


...

,zip_both_nan
zip_cc_min,0.999999


,zip_any_nan
zip_cc_max,1.0


,phone_both_nan
phone_cc_min,1.0


,phone_any_nan
phone_cc_max,1.0


<IPython.core.display.Javascript object>

In [31]:
df_merged = reduce_mem_usage(df_merged)

Memory usage after optimization is: 5.30 MB
Decreased by 48.9%


<IPython.core.display.Javascript object>

In [32]:
# DTYPES_2 = df_merged.dtypes.to_dict()

<IPython.core.display.Javascript object>

In [23]:
if not DEBUG:
    if IS_TEST:
        df_merged.to_csv(OUT_PATH + f"features_test_2_{THRESHOLD}.csv", index=False)
    else:
        df_merged.to_csv(OUT_PATH + f"features_train_2_{THRESHOLD}.csv", index=False)

<IPython.core.display.Javascript object>

In [24]:
df_merged.head()

,id_1,id_2,dist,dist1,dist2,country,cat2a,cat2b,name_pi1,name_lcs2,name_lcs,name_pi1_r1,name_lcs2_r1,name_lcs2_r2,name_lcs_r1,name_lcs_r2,name_r3,name_lcs_r4,categories_pi1,categories_lcs2,categories_lcs,categories_pi1_r1,categories_lcs2_r1,categories_lcs2_r2,categories_lcs_r1,categories_lcs_r2,categories_r3,categories_lcs_r4,address_pi1,address_pi1_r1,address_lcs2_r1,address_lcs2_r2,address_lcs_r1,address_lcs_r2,address_r3,city_NA,address_NA,phone_m10,dist_r1,dist_r2,id_cc_min,id_cc_max,name_cc_min,name_cc_max,angular_distance_min,angular_distance_l2_min,same_state,same_zip,same_city,name_len_diff,name_levenshtein,address_len_diff,address_levenshtein,url_len_diff,url_levenshtein,Nb_multiPoi_1,Nb_multiPoi_2,Nb_connect1,Nb_connect2,ratio_connect_multipoi1,ratio_connect_multipoi2,Nb_strong_connect,ratio_strong_connect_multipoi1,ratio_strong_connect_multipoi2,cat_link_score,cat_link_score_all,mean_1,mean_2,q25_1,q25_2,q50_1,q50_2,q75_1,q75_2,q90_1,q90_2,q99_1,q99_2,cat_solo_score_1,cat_solo_score_2,freq_pairing_with_other_groupedcat_1,freq_pairing_with_other_groupedcat_2,mean_ratiodist_1,mean_ratiodist_2,mean_ratiodist_pair,q25_ratiodist_1,q25_ratiodist_2,q25_ratiodist_pair,q50_ratiodist_1,q50_ratiodist_2,q50_ratiodist_pair,q75_ratiodist_1,q75_ratiodist_2,q75_ratiodist_pair,q90_ratiodist_1,q90_ratiodist_2,q90_ratiodist_pair,q99_ratiodist_1,q99_ratiodist_2,q99_ratiodist_pair,grouped_cat_link_score,grouped_cat_link_score_all,name_initial_cclcs,name_initial_lllcs,name_initial_lcs2,name_initial_lcs,name_initial_pi1,name_initial_pi2,name_initial_ld,name_initial_ljw,name_initial_dsm1,name_initial_ll1,name_initial_pi1_r1,name_initial_pi2_r1,name_initial_lcs2_r1,name_initial_lcs2_r2,name_initial_lcs_r1,name_initial_lcs_r2,name_initial_lllcs_r1,name_initial_lllcs_r2,name_initial_r3,name_initial_lcs_r4,name_initial_decode_cclcs,name_initial_decode_lllcs,name_initial_decode_lcs2,name_initial_decode_lcs,name_initial_decode_pi1,name_initial_decode_pi2,name_initial_decode_ld,name_initial_decode_ljw,name_initial_decode_dsm1,name_initial_decode_ll1,name_initial_decode_pi1_r1,name_initial_decode_pi2_r1,name_initial_decode_lcs2_r1,name_initial_decode_lcs2_r2,name_initial_decode_lcs_r1,name_initial_decode_lcs_r2,name_initial_decode_lllcs_r1,name_initial_decode_lllcs_r2,name_initial_decode_r3,name_initial_decode_lcs_r4,name_initial_decode_m5,nameC_lllcs,nameC_lcs2,nameC_lcs,nameC_pi1,nameC_pi2,nameC_ld,nameC_ljw,nameC_dsm1,nameC_ll1,nameC_pi1_r1,nameC_pi2_r1,nameC_lcs2_r1,nameC_lcs2_r2,nameC_lcs_r1,nameC_lcs_r2,nameC_lllcs_r1,nameC_lllcs_r2,nameC_r3,name_cclcs,name_lllcs,name_pi2,name_ld,name_ljw,name_dsm1,name_ll1,name_pi2_r1,name_lllcs_r1,name_lllcs_r2,name_m5,categories_pi2,categories_ld,categories_ljw,categories_dsm1,categories_ll1,categories_pi2_r1,categories_lllcs_r1,categories_lllcs_r2,address_pi2,address_ld,address_ljw,address_dsm1,address_ll1,address_pi2_r1,address_lllcs_r1,url_lcs,url_ld,url_dsm1,url_ll1,url_lcs_r2,url_r3,city_lcs,city_pi1,city_ld,city_ljw,city_dsm1,city_ll1,city_lcs2_r1,city_lcs2_r2,city_r3,state_ld,state_ljw,state_lcs2_r1,state_lcs_r2,state_r3,state_lcs_r4,state_NA,zip_pi2,zip_ljw,zip_pi1_r1,zip_lcs_r1,zip_NA,phone_lcs2,phone_pi2,phone_ljw,phone_dsm1,address_cc_min,address_cc_max,categories_cc_min,city_cc_min,city_cc_max,state_cc_min,zip_cc_min,zip_cc_max,phone_cc_min,phone_cc_max,city_group_cc_min,city_group_cc_max,state_group_cc_min,state_group_cc_max,word_c_cs,word_n_cs,id_cc_2K,id_cc_1K,id_cc_500,id_cc_200,id_cc_100,id_cc_50,id_cc_5K,id_cc_cat_2K,id_cc_cat_1K,id_cc_cat_500,id_cc_cat_200,id_cc_cat_100,id_cc_cat_50,id_cc_cat_5K,id_cc_simplcat_2K,id_cc_simplcat_1K,id_cc_simplcat_500,id_cc_simplcat_200,id_cc_simplcat_100,id_cc_simplcat_50,id_cc_simplcat_5K,name_num,address_num,langs,cat_simpl,num_in_name,nb_in_name,ratio_in_name,address_both_nan,address_any_nan,city_both_nan,city_any_nan,state_both_nan,state_any_nan,zip_both_nan,zip_any_nan,url_both_nan,url_any_nan,phone_both_nan,phone_any_nan,info_power_1,info_power_2,i

<IPython.core.display.Javascript object>

Done !